In [29]:
import os, sys, platform

# Variables

In [30]:
## Seed for the random variables
SEED_NUMBER=1;

## Kfold 
K=5;                    # Variable K of kfold
FOLD_STATUS_FILE='fold_status.json';

## Training hyperparameters
EPOCHS=200;
BATCH_SIZE=128;

## Model of network
#MODEL_TYPE  = 'model_sector4';
MODEL_TYPE  = 'model_sectortree';
#MODEL_TYPE = 'model_max'

DATASET_NAME = 'mcfer2023';
#DATASET_NAME = 'ck+48';

CPU_NAME = 'cpulab';

OUTPUT_BASE_DIR='output';

# Bibliotecas externas

In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime

In [32]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


# Biblioteca local


In [33]:
sys.path.append('library');
sys.path.append('../src');

import tools_dataset as toolsd
import tools_model   as toolsm

# If command line

In [34]:
for n in range(len(sys.argv)):
    if sys.argv[n]=='--model':
        MODEL_TYPE=sys.argv[n+1];
    if sys.argv[n]=='--epochs':
        EPOCHS=int(sys.argv[n+1]);
    if sys.argv[n]=='--batch-size':
        BATCH_SIZE=int(sys.argv[n+1]);
    if sys.argv[n]=='--dataset':
        DATASET_NAME=sys.argv[n+1];
    if sys.argv[n]=='--cpu':
        CPU_NAME=sys.argv[n+1];
    if sys.argv[n]=='--output-base-dir':
        OUTPUT_BASE_DIR=sys.argv[n+1];
        
print('     MODEL_TYPE:',MODEL_TYPE)
print('         EPOCHS:',EPOCHS)
print('     BATCH_SIZE:',BATCH_SIZE)
print('   DATASET_NAME:',DATASET_NAME)
print('       CPU_NAME:',CPU_NAME)
print('OUTPUT_BASE_DIR:',OUTPUT_BASE_DIR)

     MODEL_TYPE: model_sectortree
         EPOCHS: 200
     BATCH_SIZE: 128
   DATASET_NAME: mcfer2023
       CPU_NAME: cpulab
OUTPUT_BASE_DIR: output


# Load data

In [35]:
dataset_csv_train_file, dataset_csv_test_file, dataset_train_base_dir, input_shape, nout = toolsd.load_dataset(dataset_name=DATASET_NAME,
                                                                                                               cpu_name=CPU_NAME);

print('dataset_csv_train_file:',dataset_csv_train_file)
print(' dataset_csv_test_file:',dataset_csv_test_file)

OUTPUT_BASE_DIR=os.path.join(OUTPUT_BASE_DIR,DATASET_NAME);


dataset_csv_train_file: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/mcfer_v1.0/archive/train/training_labels.csv
 dataset_csv_test_file: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/mcfer_v1.0/archive/test/test_labels.csv


# Set seed of random variables


In [36]:
np.random.seed(SEED_NUMBER)
tf.keras.utils.set_random_seed(int(SEED_NUMBER));

# Setting the cross-validation kfold


In [37]:
from sklearn.model_selection import KFold, StratifiedKFold

# output directory
output_dir = os.path.join(OUTPUT_BASE_DIR,'skfold'+str(K)+'_'+MODEL_TYPE);

# K-fold object
kf = StratifiedKFold(n_splits = K, shuffle = True, random_state = SEED_NUMBER);

# fold status file
fold_status_path=os.path.join(output_dir,FOLD_STATUS_FILE);

print('      output_dir:',output_dir)
print('fold_status_path:',fold_status_path)

      output_dir: output/mcfer2023/skfold5_model_sectortree
fold_status_path: output/mcfer2023/skfold5_model_sectortree/fold_status.json


# Loading data of dataset

In [38]:
# Load filenames and labels
train_data = pd.read_csv(dataset_csv_train_file);
print(train_data)

# Setting labels
Y   = train_data[['label']];

L=np.shape(Y)[0];

                                    filename      label
0           angry/angry-contempt/im344_2.png      angry
1      disgusted/awful/affectnet_0036766.png  disgusted
2       fearful/attention-fearful/im32_1.png    fearful
3         happy/attention-happy/im1325_2.png      happy
4                neutral/attention/im2_1.png    neutral
...                                      ...        ...
24215           happy/light-smile/im4742.png      happy
24216           happy/light-smile/im4809.png      happy
24217         happy/light-smile/im4921_1.png      happy
24218           happy/light-smile/im5535.png      happy
24219           happy/light-smile/im5545.png      happy

[24220 rows x 2 columns]


# Data augmentation configuration

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg    = ImageDataGenerator(rescale=1./255,
                            rotation_range = 10,
                            width_shift_range= 0.07,
                            height_shift_range= 0.07,
                            horizontal_flip=True,
                            shear_range=1.25,
                            zoom_range = [float(0.9), float(1.1)] 
                            )

idg_val= ImageDataGenerator(rescale=1./255 )



# Auxiliar function

In [40]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

# Creating output directory

In [41]:

try: 
    os.makedirs(output_dir) 
except: 
    pass


# Cross-validation

In [ ]:
import tools_model as mpp
import matplotlib.pyplot as plt


list_train_index=[];
list_val_index=[];
for train_index, val_index in kf.split(np.zeros(L),Y):
    list_train_index.append(train_index);
    list_val_index.append(val_index);

import json
data_fold =  {'VALIDATION_ACCURACY': [],'VALIDATION_LOSS': [] };

fold_var=1;

if os.path.isfile(fold_status_path):
    # Read JSON file
    with open(fold_status_path) as data_file:
        data_fold = json.load(data_file)
        fold_var=len(data_fold['VALIDATION_ACCURACY'])+1;
    
while fold_var<=K:
    training_data   = train_data.iloc[list_train_index[fold_var-1]]
    validation_data = train_data.iloc[list_val_index[fold_var-1]]

    print('\nFold',fold_var,'of',K);
    print('length train:',len(list_train_index[fold_var-1]),'elements');
    print('length val  :',len(list_val_index[fold_var-1]),'elements');

    # CREATE NEW MODEL
    
    model, target_size = mpp.create_model(  file_of_weight='',
                                            model_type=MODEL_TYPE,
                                            input_shape=input_shape,
                                            nout=nout)
    
    model.summary()
    
    train_data_generator = idg.flow_from_dataframe(training_data, 
                                                   directory = dataset_train_base_dir,
                                                   target_size=target_size,
                                                   x_col = "filename", 
                                                   y_col = "label",
                                                   batch_size=BATCH_SIZE,
                                                   class_mode="categorical",
                                                   shuffle = True);
    
    valid_data_generator  = idg_val.flow_from_dataframe(validation_data, 
                                                    directory = dataset_train_base_dir,
                                                    target_size=target_size,
                                                    x_col = "filename", 
                                                    y_col = "label",
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="categorical",
                                                    shuffle = True)
    
    #STEPS_BY_EPOCHS=len(train_data_generator);
    

    
    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])
    
    # CREATE CALLBACKS
    best_model_file=os.path.join(output_dir,get_model_name(fold_var));
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_file, 
                                                    save_weights_only=True,
                                                    monitor='val_categorical_accuracy', 
                                                    save_best_only=True, 
                                                    verbose=1);
    
    log_dir = os.path.join(output_dir,"logs","fit" , datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(train_data_generator,
                        #steps_per_epoch=STEPS_BY_EPOCHS,
                        epochs=EPOCHS,
                        validation_data=valid_data_generator,
                        callbacks=[checkpoint,tensorboard_callback],
                        verbose=1
                       );
    print('***** Saving the history *****')

    #PLOT HISTORY
    mpp.save_model_history(history,os.path.join(output_dir,"historical_"+str(fold_var)+".csv"), labels=['categorical_accuracy','loss'],show=False);
    
    print('***** Loding the best model file ... *****')
    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights(best_model_file);
    
    print('***** Evaluating the model ... *****')
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))
    print(results,"\n\n");
    
    data_fold['VALIDATION_ACCURACY'].append(results['categorical_accuracy'])
    data_fold['VALIDATION_LOSS'].append(results['loss'])
    
    # Data fold
    with open(fold_status_path, 'w') as f:
        json.dump(data_fold, f);

    tf.keras.backend.clear_session()
    
    fold_var += 1


Fold 1 of 5
length train: 19376 elements
length val  : 4844 elements
Loaded model_sector4
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_66 (Conv2D)          (None, 224, 224, 16)      5824      
                                                                 
 conv2d_67 (Conv2D)          (None, 224, 224, 4)       5188      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 112, 112, 4)      0         
 g2D)                                                            
                                                                 
 conv2d_68 (Conv2D)          (None, 112, 112, 16)      5200      
                                                                 
 conv2d_69 (Conv2D)          (None, 112, 112, 4)       3140      
                                                                 
 max_pooling2d_14 (MaxPoolin

In [ ]:
fpath=os.path.join(output_dir,"final_stats.csv");
mean_val_acc=mpp.save_model_stat_kfold(data_fold, fpath);

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.txt'));

print(mean_val_acc)